In [25]:
import sys
import os
import pandas as pd

PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    
from src.validation.balance_checks import add_balance_consistency_flag
from src.preprocessing.feature_engineering import add_time_features
from src.categorization.recurring_detection import detect_recurring_merchants
from src.analysis.recurring_split import split_fixed_habitual
from src.analysis.spending_aggregation import compute_spending_aggregates
from src.feasibility.savings_engine import evaluate_savings_feasibility
from src.genai.explanation_engine import generate_savings_explanation


In [26]:
df = pd.read_csv("C:/Users/harsh/Spending Behavior Analyzer/data/raw/synthetic_hdfc_john_doe.csv")

In [27]:
df = add_balance_consistency_flag(df)
df = add_time_features(df)
if "date" not in df.columns:
    raise ValueError("Expected 'date' column not found")

monthly_income = (
    df[df["credit_amount"] > 0]
    .groupby("year_month")["credit_amount"]
    .sum()
    .mean()
)

target_savings = float(input("Enter target savings"))

recurring_merchants = detect_recurring_merchants(df)

recurring_split = split_fixed_habitual(df, recurring_merchants)

fixed_monthly = recurring_split["fixed_monthly_avg"]
habitual_monthly = recurring_split["habitual_monthly_avg"]

spend_stats = compute_spending_aggregates(df)

result = evaluate_savings_feasibility(
    monthly_income=monthly_income,
    fixed_monthly=fixed_monthly,
    habitual_monthly=habitual_monthly,
    total_spend=spend_stats["monthly_total_spend"],
    target_savings=target_savings
)

explanation = generate_savings_explanation(result)
print(explanation)


Based on your financial information, your target savings of ₹4500.0 per month is **feasible without needing to make any changes to your current spending habits**.

You currently have ₹30282.0 available to save each month after covering all your average monthly expenses. This amount is significantly higher than your target savings of ₹4500.0, leaving you with a substantial surplus.

Therefore, cutting down on habitual expenses is not necessary to meet this specific savings goal.

To make the most of this positive situation, we recommend setting up an automatic transfer for your ₹4500.0 savings each month. You may also want to review how you could utilize the additional surplus you have available for other financial goals, such as building an emergency fund, investing, or saving for larger purchases.


In [28]:
df

,account_holder,date,description,debit_amount,credit_amount,balance,prev_balance,expected_balance,balance_consistent,is_weekend,year_month
0,John Doe,2024-01-01,SALARY CREDIT,0,60000,60000,NaN,NaN,True,False,2024-01
1,John Doe,2024-01-01,NETFLIX,799,0,59201,60000.0,59201.0,True,False,2024-01
2,John Doe,2024-01-02,MOBILE RECHARGE,499,0,58702,59201.0,58702.0,True,False,2024-01
3,John Doe,2024-01-03,HOUSE RENT,15000,0,43702,58702.0,43702.0,True,False,2024-01
4,John Doe,2024-01-04,NETFLIX,799,0,42903,43702.0,42903.0,True,False,2024-01
...,...,...,...,...,...,...,...,...,...,...,...
85,John Doe,2024-06-22,OLA,1285,0,184124,185409.0,184124.0,True,True,2024-06
86,John Doe,2024-06-23,NETFLIX,799,0,183325,184124.0,183325.0,True,True,2024-06
87,John Doe,2024-06-24,ZOMATO,418,0,182907,183325.0,182907.0,True,False,2024-06
88,John Doe,2024-06-25,ZOMATO,803,0,182104,182907.0,182104.0,True,False,2024-06


In [29]:
spend_stats

{'monthly_total_spend': 29718.0,
 'weekend_total': 54918,
 'weekday_total': 123390,
 'weekend_avg': 2196.72,
 'weekday_avg': 2091.35593220339,
 'weekend_txn_count': 25,
 'weekday_txn_count': 59}

In [30]:
result

{'monthly_income': 60000.0,
 'fixed_monthly': 23106.833333333332,
 'habitual_monthly': 5780.833333333333,
 'monthly_total_spend': 29718.0,
 'available_to_save': 30282.0,
 'target_savings': 4500.0,
 'shortfall': -25782.0,
 'feasibility': 'Feasible without changes',
 'fixed_load_pct': 38.51138888888889,
 'habitual_load_pct': 9.634722222222223,
 'max_possible_behavioral_cut': 5780.833333333333}

In [31]:
spend_stats = compute_spending_aggregates(df)
print(spend_stats)


{'monthly_total_spend': 29718.0, 'weekend_total': 54918, 'weekday_total': 123390, 'weekend_avg': 2196.72, 'weekday_avg': 2091.35593220339, 'weekend_txn_count': 25, 'weekday_txn_count': 59}
